In [ ]:
!pip install --quiet apache_beam


In [3]:
import apache_beam as beam
from apache_beam.dataframe import convert
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [4]:
def validate_words(row, key):
  cv = CountVectorizer()
  cv_fit = cv.fit_transform( [row.get(key)] )
  words = cv.get_feature_names_out()
  check =  any(item in ['good', 'GOOD', 'goOd', 'gOod', 'gOOD', 'Good'] for item in words.tolist())
  if check is True:
    row['positive_review'] = 1
    return row
  else :
    row['positive_review'] = 0
    return row

In [6]:
with beam.Pipeline() as p:
  df = pd.read_csv('/content/litle_movie_review.csv')  
  # raw_csv = (p 
  #           #  | '1. read csv' >> beam.io.ReadFromText('/content/movie_review.csv')
  #            | beam.dataframe.io.read_csv('/content/movie_review.csv', usecols = ['cid', 'review_str' , 'id_review'])
  #           #  | beam.Map(lambda x: x.split(',') )

  data_splited = (
      convert.to_pcollection(df, pipeline = p)
      | beam.Map(lambda x: dict(x._asdict()))

  )

  cv_fit = (
      data_splited 
      | beam.Map(lambda x : validate_words(x, 'review_str'))
      | beam.Map(print)
      | 'Write to files' >> beam.io.WriteToText(
          '/content/litle_movies_revies_output.csv')
  )    

{'cid': 13756, 'review_str': "Once again Mr. Costner has dragged out a movie for far longer than necessary. Aside from the terrific sea rescue sequences, of which there are very few I just did not care about any of the characters. Most of us have ghosts in the closet, and Costner's character are realized early on, and then forgotten until much later, by which time I did not care. The character we should really care about is a very cocky, overconfident Ashton Kutcher. The problem is he comes off as kid who thinks he's better than anyone else around him and shows no signs of a cluttered closet. His only obstacle appears to be winning over Costner. Finally when we are well past the half way point of this stinker, Costner tells us all about Kutcher's ghosts. We are told why Kutcher is driven to be the best with no prior inkling or foreshadowing. No magic here, it was all I could do to keep from turning it off an hour in.", 'id_review': 1, 'positive_review': 0}
{'cid': 15738, 'review_str': 